In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from pymol import cmd

In [2]:
# Launch PyMOL with the "-R" option (pymol -R), 
# open a Python 3 notebook, then execute 
# remote control of PyMOL (RPC) with XML-RPC:
# https://pymolwiki.org/index.php/Jupyter
# https://pymolwiki.org/index.php/RPC

import xmlrpc.client as xmlrpclib
cmd = xmlrpclib.ServerProxy('http://localhost:9123')

#from IPython import display
import os
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
pymol_session = '/home/evasmo/work/other/kcn_mono_oligo_interface_contacts.pse'

In [4]:
sequences = {'KCNN4':
                {'native': 'RRRKRLLEQEKSLAGWALVLAGTGIGLMVLHAEMLWFGGCSWALYLFLVKCTISISTFLLLCLIVAFHAKEVQLFMTDNGLRDWRVALTGRQAAQIVLELVVCGLHPAPVRPGFLGQGEALLSLAMLLRLYLVPRAVLLRSGVLLNASYRSIGALNQVRFRHWFVAKLYMNTHPGRLLLGLTLGLWLTTAWVLSVAERQAVNATGHLSDTLWLIPITFLTIGYGDVVPGTMWGKIVCLCTGVMGVCCTALLVAVVARKLEFNKAEKH', 
                 'qty':    'RRRKRLLEQEKSLAGWAQTQAGTGTGQMTQHAEMQWYGGCSWALYLYQTKCTTSTSTYQQQCQTTAYHAKEVQLFMTDNGLRDWRVALTGRQAAQTTQEQTTCGQHPAPTRPGYQGQGEAQQSQAMQQRQYQTPRAVLLRSGVLLNASYRSIGALNQVRFRHWFVAKLYMNTHPGRQQQGQTQGQWQTTAWTQSTAERQAVNATGHLSDTLWLIPITFLTIGYGDVVPGTMWGKTTCQCTGTMGTCCTAQQTATTARKLEFNKAEKH', 
                 'cryoem': 'RRRKRLLEQEKSLAGWALVLAGTGIGLMVLHAEMLWFGGCSWALYLFLVKCTISISTFLLLCLIVAFHAKEVQLFMTDNGLRDWRVALTGRQAAQIVLELVVCGLHPAPVRPGFLGQGEALLSLAMLLRLYLVPRAVLLRSGVLLNASYRSIGALNQVRFRHWFVAKLYMNTHPGRLLLGLTLGLWLTTAWVLSVAERQAVNATGHLSDTLWLIPITFLTIGYGDVVPGTMWGKIVCLCTGVMGVCCTALLVAVVARKLEFNKAEKH'},
             'KCNJ11':
                {'native': 'KWPHTLLIFTMSFLCSWLLFAMAWWLIAFAHGDLAPSEGTAEPCVTSIHSFSSAFLFSIEVQVTIGFGGRMVTEECPLAILILIVQNIVGLMINAIMLGCIFMKTAQAHRR',
                 'qty':    'KWPHTQQTYTMSYQCSWQQYAMAWWQTAFAHGDLAPSEGTAEPCVTSIHSYSSAYQYSTETQTTTGYGGRMVTEECPLATQTQTTQNTTGQMTNATMQGCTFMKTAQAHRR',
                 'cryoem': 'KWPHTLLIFTMSFLCSWLLFAMAWWLIAFAHGDLAPSEGTAEPCVTSIHSFSSAFLFSIEVQVTIGFGGRMVTEECPLAILILIVQNIVGLMINAIMLGCIFMKTAQAHRR'}        
    }

In [5]:
cmd.load(pymol_session)
cmd.do('set internal_gui_width, 400')

In [6]:
objects = cmd.get_object_list()

In [7]:
kcnn4_objects = [o for o in objects if 'KCNN4' in o]
kcnj11_objects = [o for o in objects if 'KCNJ11' in o]

In [8]:
systems = {'KCNN4': kcnn4_objects,
           'KCNJ11': kcnj11_objects}

In [9]:
def get_overlap(list1, list2):
    overlap = set(list1) & set(list2)
    return list(overlap), len(overlap), len(list1), len(list2)

def find_diff(str1, str2):
    # Ensure both strings are of the same length to avoid index errors
    min_len = min(len(str1), len(str2))
    
    # Compare characters in both strings and note positions where they differ, starting from 1
    diff_positions = [i + 1 for i in range(min_len) if str1[i] != str2[i]]
    
    # If the strings are of different lengths, append the extra characters' positions
    if len(str1) > len(str2):
        diff_positions.extend(range(min_len + 1, len(str1) + 1))
    elif len(str2) > len(str1):
        diff_positions.extend(range(min_len + 1, len(str2) + 1))
    
    return diff_positions

In [10]:
def interface(protein):

    cmd.show('cartoon')
    
    [cmd.align(o, f'*_cryoem_{protein}') for o in systems[protein] if not o.endswith(protein)]
    
    cmd.do(f'''select {protein}_cryoem_oligo_cont, (byres all within 5 of {protein}_cryoem*) and (not chain A and *_cryoem_{protein})
    select {protein}_qty_solv_oligo_cont, (byres all within 5 of {protein}_qty_solv*) and (not chain A and *_cryoem_{protein})
    select {protein}_qty_mem_oligo_cont, (byres all within 5 of {protein}_qty_400*) and (not chain A and *_cryoem_{protein})
    select {protein}_native_oligo_cont, (byres all within 5 of {protein}_native*) and (not chain A and *_cryoem_{protein})
    
    select {protein}_cryoem_mono_cont, (byres all within 5 of {protein}_cryoem_oligo_cont) and ({protein}_cryoem_400*)
    select {protein}_qty_solv_mono_cont, (byres all within 5 of {protein}_qty_solv_oligo_cont) and ({protein}_qty_solvent*)
    select {protein}_qty_mem_mono_cont, (byres all within 5 of {protein}_qty_mem_oligo_cont) and ({protein}_qty_400*)
    select {protein}_native_mono_cont, (byres all within 5 of {protein}_native_oligo_cont) and ({protein}_native_400*)''')
    
    selections = [
        f'{protein}_cryoem_oligo_cont', f'{protein}_qty_solv_oligo_cont', f'{protein}_qty_mem_oligo_cont', f'{protein}_native_oligo_cont', 
        f'{protein}_cryoem_mono_cont', f'{protein}_qty_solv_mono_cont', f'{protein}_qty_mem_mono_cont', f'{protein}_native_mono_cont'
    ]
    
    selected_residues = {}
    
    for selection in selections:
        model = cmd.get_model(selection)
        temp = []
        for atom in model['atom']:
        #if atom['name'] == 'CA': 
            resi = atom['resi_number']
            resn = atom['resn']
            temp.append((resn, resi))
            
        selected_residues[selection] = list(set(temp))
    
    # Overlap between structural interfaces of different systems (how similar are structiral interfaces)
    cryoem_cryoem = get_overlap(selected_residues[f'{protein}_cryoem_oligo_cont'], selected_residues[f'{protein}_cryoem_oligo_cont'])
    cryoem_qty_solv = get_overlap(selected_residues[f'{protein}_cryoem_oligo_cont'], selected_residues[f'{protein}_qty_solv_oligo_cont'])
    cryoem_qty_mem = get_overlap(selected_residues[f'{protein}_cryoem_oligo_cont'], selected_residues[f'{protein}_qty_mem_oligo_cont'])
    #cryoem_native = get_overlap(selected_residues[f'{protein}_cryoem_oligo_cont'], selected_residues[f'{protein}_native_oligo_cont'])
    
    assert cryoem_cryoem[1] == cryoem_cryoem[2] == cryoem_cryoem[3]
    
    qty_mut_pos = find_diff(sequences[protein]['native'], sequences[protein]['qty'])
    
    # Overlap between QTY mutations and the interfaces (how many mutations are at the interface)
    cryoem_cryoem_inter = get_overlap(qty_mut_pos, [i[1] for i in cryoem_cryoem[0]])
    cryoem_qty_solv_inter = get_overlap(qty_mut_pos, [i[1] for i in cryoem_qty_solv[0]])
    cryoem_qty_mem_inter = get_overlap(qty_mut_pos, [i[1] for i in cryoem_qty_mem[0]])
    #cryoem_native_inter = get_overlap(qty_mut_pos, [i[1] for i in cryoem_native[0]])
    
    print('How many monomer residues are at the interface in the Cryo-EM structure?') # All interface residues
    print(f"Cryo-EM: {cryoem_cryoem[3]} out of {len(sequences[protein]['cryoem'])} residues are at the interface ({round((cryoem_cryoem[3] / len(sequences[protein]['cryoem'])) * 100, 2)}%)\n")
    
    print('How many monomer residues are at the interface in the QTY structure?') # All interface residues
    print(f"QTY in water: {cryoem_qty_solv[3]} out of {len(sequences[protein]['qty'])} residues are at the interface ({round((cryoem_qty_solv[3] / len(sequences[protein]['qty'])) * 100, 2)}%)")
    print(f"QTY in membrane: {cryoem_qty_mem[3]} out of {len(sequences[protein]['qty'])} residues are at the interface ({round((cryoem_qty_mem[3] / len(sequences[protein]['qty'])) * 100, 2)}%)\n")
    
    print('How many Cryo-EM monomer interface residues were mutated to QTY?') # Mutated interface residues
    print(f"Cryo-EM: {cryoem_cryoem_inter[1]} out of {cryoem_cryoem_inter[3]} residues were mutated ({round((cryoem_cryoem_inter[1] / cryoem_cryoem_inter[3]) * 100, 2)}% out of Cryo-EM interface and {round((cryoem_cryoem_inter[1] / len(qty_mut_pos)) * 100, 2)}% out of all QTY mutations)\n")
    
    print('How many mutations are at the interface in QTY in water and membrane?') # Mutated interface residues
    print(f"QTY in water: {cryoem_qty_solv_inter[1]} out of {cryoem_qty_solv_inter[2]} ({round((cryoem_qty_solv_inter[1] / cryoem_qty_solv_inter[3]) * 100, 2)}% out of QTY interface and {round((cryoem_qty_solv_inter[1] / len(qty_mut_pos)) * 100, 2)}% out of all QTY mutations)")
    print(f"QTY in membrane: {cryoem_qty_mem_inter[1]} out of {cryoem_qty_mem_inter[2]} ({round((cryoem_qty_mem_inter[1] / cryoem_qty_mem_inter[3]) * 100, 2)}% out of QTY interface and {round((cryoem_qty_mem_inter[1] / len(qty_mut_pos)) * 100, 2)}% out of all QTY mutations)")
    
    cmd.do(f'''select {protein}_qty_solv_inter, resi {'+'.join([str(i) for i in cryoem_qty_solv_inter[0]])} and {protein}_qty_solvent*
    select {protein}_qty_mem_inter, resi {'+'.join([str(i) for i in cryoem_qty_mem_inter[0]])} and {protein}_qty_400*
    select {protein}_qty_solv_all_mut, resi {'+'.join([str(i) for i in qty_mut_pos])} and {protein}_qty_solvent*
    select {protein}_qty_mem_all_mut, resi {'+'.join([str(i) for i in qty_mut_pos])} and {protein}_qty_400*
    ''')
    
    cmd.show('spheres', f'name CA and ({protein}_qty_solv_all_mut or {protein}_qty_mem_all_mut)')
    cmd.do('set sphere_scale, 0.5')
    #cmd.do('''color red, elem O
    #color slate, elem N
    #color gray98, elem H''')
    cmd.color('black', f'{protein}_qty_solv_inter or {protein}_qty_mem_inter')
    cmd.do('set cartoon_transparency, 0.5')

In [11]:
interface('KCNN4')

How many monomer residues are at the interface in the Cryo-EM structure?
Cryo-EM: 77 out of 267 residues are at the interface (28.84%)

How many monomer residues are at the interface in the QTY structure?
QTY in water: 61 out of 267 residues are at the interface (22.85%)
QTY in membrane: 73 out of 267 residues are at the interface (27.34%)

How many Cryo-EM monomer interface residues were mutated to QTY?
Cryo-EM: 9 out of 77 residues were mutated (11.69% out of Cryo-EM interface and 15.0% out of all QTY mutations)

How many mutations are at the interface in QTY in water and membrane?
QTY in water: 5 out of 60 (12.2% out of QTY interface and 8.33% out of all QTY mutations)
QTY in membrane: 5 out of 60 (9.43% out of QTY interface and 8.33% out of all QTY mutations)


In [12]:
interface('KCNJ11')

How many monomer residues are at the interface in the Cryo-EM structure?
Cryo-EM: 91 out of 111 residues are at the interface (81.98%)

How many monomer residues are at the interface in the QTY structure?
QTY in water: 79 out of 111 residues are at the interface (71.17%)
QTY in membrane: 86 out of 111 residues are at the interface (77.48%)

How many Cryo-EM monomer interface residues were mutated to QTY?
Cryo-EM: 7 out of 91 residues were mutated (7.69% out of Cryo-EM interface and 21.21% out of all QTY mutations)

How many mutations are at the interface in QTY in water and membrane?
QTY in water: 5 out of 33 (8.33% out of QTY interface and 15.15% out of all QTY mutations)
QTY in membrane: 6 out of 33 (8.45% out of QTY interface and 18.18% out of all QTY mutations)
